# Forecasting Walmart sales with Treeffuser

In this tutorial we show how to use Treeffuser to model and forecast Walmart sales using the M5 forecasting dataset from Kaggle.

## Getting started


To get started, we first install `treeffuser` and import the relevant libraries.

In [ ]:
!pip install treeffuser

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from pathlib import Path

from tqdm import tqdm
from treeffuser import Treeffuser

# load autoreload extension
%load_ext autoreload
%autoreload 2

Next, create a Kaggle account and download the data from https://www.kaggle.com/competitions/m5-forecasting-accuracy/data.

If you're running this notebook in Colab, manually upload the necessary files (`calendar.csv`, `sales_train_validation.csv`, `sell_prices.csv`) to Colab by clicking the `Files` tab on the left sidebar and selecting `Upload`. Move the files into a new folder named `m5`. Once uploaded, the notebook will be able to read and process the data.

If you're running this on your local machine, you can also use Kaggle's [command-line tool](https://www.kaggle.com/docs/api) and run the following from the command line:

```bash
cd ./m5 # path to folder where you want to save the data
kaggle competitions download -c m5-forecasting-accuracy
```

Use your favorite tool to unzip the archive. In Linux/macOS,

```bash
unzip m5-forecasting-accuracy.zip
```

We'll be using the following files: `calendar.csv`, `sales_train_validation.csv`, and `sell_prices.csv`.


<!-- - `calendar.csv` - Contains information about the dates on which the products are sold.
- `sales_train_validation.csv` - Contains the historical daily unit sales data per product and store `[d_1 - d_1913]`.
- `sell_prices.csv` - Contains information about the price of the products sold per store and date. -->
<!-- - `sales_train_evaluation.csv`- Includes sales [`d_1 - d_1941]` (labels used for the Public leaderboard). -->
<!-- - `sample_submission.csv` - The correct format for submissions. Reference the Evaluation tab for more info. -->

In [2]:
# import data
data_path = "./m5"  # change with path where you extracted the data archive

calendar_df = pd.read_csv(Path(data_path) / "calendar.csv")
sales_train_df = pd.read_csv(Path(data_path) / "sales_train_validation.csv")
sell_prices_df = pd.read_csv(Path(data_path) / "sell_prices.csv")

# add explicit columns for the day, month, year for ease of processing
calendar_df["date"] = pd.to_datetime(calendar_df["date"])
calendar_df["day"] = calendar_df["date"].dt.day
calendar_df["month"] = calendar_df["date"].dt.month
calendar_df["year"] = calendar_df["date"].dt.year

## The data

### Preprocessing
`sell_prices_df` contains the prices of each item in each store at a given time. The `wm_yr_wk` is a unique identifier for the time.

In [ ]:
sell_prices_df.head()

`calendar_df` contains information about the dates on which the products were sold.

In [ ]:
calendar_df.head()

`sales_train_df` contains the number of units sold for an item in each department and store. The sales are grouped by day: for example, the `d_1907` column has the number of units sold on the 1907-th day.

In [ ]:
sales_train_df.head()

To align the sales data with the other DataFrames, we convert `sales_train_df` to a long format. We collapse the daily sales columns `d_{i}` into a single `sales` column, with an  additional `day` column indicating the day corresponding to each sales entry.

In [ ]:
def convert_sales_data_from_wide_to_long(sales_df_wide):
    index_vars = ["item_id", "dept_id", "cat_id", "store_id", "state_id"]
    sales_df_long = pd.wide_to_long(
        sales_df_wide.iloc[:100, 1:],
        i=index_vars,
        j="day",
        stubnames=["d"],
        sep="_",
    ).reset_index()

    sales_df_long = sales_df_long.rename(columns={"d": "sales", "day": "d"})

    sales_df_long["d"] = "d_" + sales_df_long["d"].astype(
        "str"
    )  # restore "d_{i}" format for day
    return sales_df_long


sales_train_df_long = convert_sales_data_from_wide_to_long(sales_train_df)
sales_train_df_long.head()

In [ ]:
plt.hist(
    sales_train_df_long["sales"],
    bins=np.arange(0, 10 + 1.5) - 0.5,
    range=[0, 10],
    density=True,
)
plt.xticks(range(10))
plt.ylabel("relative frequency")
plt.title("number of sales over the entire timespan")

### Train and test sets

The dataset comprises sales data of 100 items over 1,913 days. For simplicity, we select the data from the first 365 days and discard the rest.

In [ ]:
print(f"n_items = {len(sales_train_df_long['item_id'].unique())}")
print(f"n_days = {len(sales_train_df_long['d'].unique())}")

sales_train_df_long["day_number"] = sales_train_df_long["d"].str.extract("(\d+)").astype(int)
data = sales_train_df_long[sales_train_df_long["day_number"] <= 365].copy()

We compute the lags of the previous 30 days and merge the sales, calendar, and price data together.

In [ ]:
n_lags = 30

# sort data before computing lags
data_index_vars = ["item_id", "dept_id", "cat_id", "store_id", "state_id"]
data.sort_values(data_index_vars + ["day_number"], inplace=True)

for lag in range(1, n_lags + 1):
    data[f"sales_lag_{lag}"] = data.groupby(by=data_index_vars)["sales"].shift(lag)

data = data.merge(calendar_df).merge(sell_prices_df)

data.head()

Finally, for each item, we take the first 300 days as train data and use the remaining 65 data as test data for posterior predictive checks.

In [ ]:
is_train = data["day_number"] <= 300
data = data.drop(columns=["day_number", "day"])

y_name = "sales"
x_names = [name for name in data.columns if name != y_name]

X_train, y_train = data[is_train][x_names], data[is_train][y_name]
X_test, y_test = data[~is_train][x_names], data[~is_train][y_name]

print(X_train.shape)
print(X_test.shape)

The features of the combined datasets are mostly categorical. We save the column indices in `cat_idx` as they will come in handy later.

In [ ]:
cat_column_names = [
    "item_id",
    "dept_id",
    "cat_id",
    "store_id",
    "state_id",
    "d",
    "date",
    "wm_yr_wk",
    "weekday",
    "wday",
    "month",
    "year",
    "event_name_1",
    "event_type_1",
    "event_name_2",
    "event_type_2",
    "snap_CA",
    "snap_TX",
    "snap_WI",
]
cat_idx = [data.columns.get_loc(col) for col in cat_column_names]
cat_idx

## Probabilistic predictions with Treeffuser

We regress the sales on the following covariates.

In [ ]:
print(", ".join(map(str, X_train.columns)))

Currently, Treeffuser supports only numpy.ndarray data with numerical values. Therefore, we convert the categorical columns into numerical labels and then convert the train and test data into numpy.ndarray.

In [13]:
X_train[cat_column_names] = X_train[cat_column_names].apply(
    lambda col: pd.Categorical(col).codes
)

X_train, y_train = X_train.to_numpy(), y_train.to_numpy()
X_test, y_test = X_test.to_numpy(), y_test.to_numpy()

We are now ready to fit the data. We use `cat_idx` to tell Treeffuser which columns are categorical.

In [ ]:
model = Treeffuser()
model.fit(X_train, y_train, cat_idx=cat_idx)